In [ ]:
epochs = 10
# We don't use the whole dataset for efficiency purpose, but feel free to increase these numbers
n_train_items = 640
n_test_items = 640

# Teil X - Sicheres Trainieren und Evaluieren mit MNIST

Wenn es darum geht Machine Learning als Service (MLaaS) aufzubauen, müssen Unternehmen manchmal auf Daten eines anderen Partners zurückgreifen um ihr Model trainieren zu können. Im Gesundheits- oder Finanzbereich sind solche Daten jedoch extrem kritisch: während die Model Parameter einem Unternehmen gehören, sind die persönlichen Daten streng reguliert. 

In diesem Kontext stellt das vertrauliche Trainieren auf verschlüsselten Daten und Modelen eine mögliche Lösung dar. Dies garantiert, dass Unternehmen keinen direkten Zugang zu den Patientenakten bekommen und ebenso, dass Gesundheitsorganisationen keinen Einblick in das Model erhalten zu dem sie beitragen.  
Es existieren unterschiedliche Verschlüsselungs Schemata, die für die Berechnungen mit verschlüsselten Daten verwendet werden können. Mit dazu gehören "Secure Multi-Party Computation" (SMPC), "Homomorphic Encryption" (FHE/SHE) und "Functional Encryption" (FE). Hier wird der Fokus auf "Secure Multi-Party Computation" mit seinem additiven Aufteilen gelegt. Aufgebaut ist alles auf den Crypto-Protokollen SecureNN und SPDZ.

Die Ausgangslage für dieses Tutorial sieht wie folgt aus: es gibt einen Server, der sein Model auf den Daten von $n$ Helfern trainieren möchte. Der Server teilt sein Model dazu auf und versendet es an die Helfer. Die Helfer teilen gleichsam ihre Daten sicher auf und verteilen sie untereinander. In diesem Tutorial werden zwei Helfer (`alice` und `bob`) für den Aufbau verwendet. Nach dem Austausch besitzt jeder Helfer einen Anteil seiner eigenen Daten, einen Anteil der Daten des anderen Helfers und auch einen Anteil des Models. Das Training kann nun mit den jeweiligen Crypto-Protokollen gestartet werden. Sobald das Model trainiert ist, können alle Anteile an den Server zurückgesendet und entschlüsselt werden.  
Das Vorgehen kann in der nachfolgenden Grafik nachvollzogen werden:

![SMPC Illustration](https://github.com/OpenMined/PySyft/raw/11c85a121a1a136e354945686622ab3731246084/examples/tutorials/material/smpc_illustration.png)

Als Beispiel für diesen Prozess wird angenommen, dass sowohl Alice wie auch Bob jeweils einen Teil des MNIST Datensatzes besitzen und darauf ein Model zum Klassifizieren der Ziffern trainiert werden soll. 

Autoren:
- Théo Ryffel - Twitter: [@theoryffel](https://twitter.com/theoryffel) · GitHub: [@LaRiffle](https://github.com/LaRiffle)

Übersetzer:
- Jan Moritz Behnken - Github: [@JMBehnken](https://github.com/JMBehnken)

# 1. Verschlüsseltes Trainings Beispiel mit MNIST

## Importe und Trainings Konfiguration

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import time

Diese Klasse beinhaltet alle Hyper-Parameter für das Training. Anzumerken ist, dass diese alle öffentlich bereitgestellt werden.

In [ ]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 64
        self.epochs = epochs
        self.lr = 0.02
        self.seed = 1
        self.log_interval = 1 # Log info at each batch
        self.precision_fractional = 3

args = Arguments()

_ = torch.manual_seed(args.seed)

Es folgen die PySyft Importe. Eine Verbindung zu den beiden Helfern `alice` und `bob` wird hergestellt und auch ein `crypto_provider` für den sicheren Austausch wird erstellt.

In [ ]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch to add extra functionalities like Federated and Encrypted Learning

# simulation functions
def connect_to_workers(n_workers):
    return [
        sy.VirtualWorker(hook, id=f"worker{i+1}")
        for i in range(n_workers)
    ]
def connect_to_crypto_provider():
    return sy.VirtualWorker(hook, id="crypto_provider")

workers = connect_to_workers(n_workers=2)
crypto_provider = connect_to_crypto_provider()

## Zugang zu den sicher geteilten Daten gewähren

Hier wird eine nützliche Funktion verwendet, die folgendes Verhalten simuliert: es wird angenommen, dass der MNIST Datensatz zu teilen jeweils von den einzelnen Helfern gespeichert wird. Diese Helfer teilen ihre Daten in Batches auf und teilen diese wiederum sicher untereinander auf. Das finale Objekt ist ein Iterator über die sicher geteilten Batches und wird **vertraulicher Daten-Loader** genannt. Anzumerken ist, dass die lokale Machine dabei zu keiner Zeit Zugang zu den Rohdaten hat.

Wie normalerweise auch, wird ein Trainings-Datensatz und ein Test-Datensatz bereit gestellt. In diesen sind die Eingabe-Daten und auch die Ziel-Variablen jeweils verschlüsselt.

In [ ]:
def get_private_data_loaders(precision_fractional, workers, crypto_provider):
    
    def one_hot_of(index_tensor):
        """
        Transform to one hot tensor
        
        Example:
            [0, 3, 9]
            =>
            [[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
             [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]]
            
        """
        onehot_tensor = torch.zeros(*index_tensor.shape, 10) # 10 classes for MNIST
        onehot_tensor = onehot_tensor.scatter(1, index_tensor.view(-1, 1), 1)
        return onehot_tensor
        
    def secret_share(tensor):
        """
        Transform to fixed precision and secret share a tensor
        """
        return (
            tensor
            .fix_precision(precision_fractional=precision_fractional)
            .share(*workers, crypto_provider=crypto_provider, requires_grad=True)
        )
    
    transformation = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transformation),
        batch_size=args.batch_size
    )
    
    private_train_loader = [
        (secret_share(data), secret_share(one_hot_of(target)))
        for i, (data, target) in enumerate(train_loader)
        if i < n_train_items / args.batch_size
    ]
    
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True, transform=transformation),
        batch_size=args.test_batch_size
    )
    
    private_test_loader = [
        (secret_share(data), secret_share(target.float()))
        for i, (data, target) in enumerate(test_loader)
        if i < n_test_items / args.test_batch_size
    ]
    
    return private_train_loader, private_test_loader
    
    
private_train_loader, private_test_loader = get_private_data_loaders(
    precision_fractional=args.precision_fractional,
    workers=workers,
    crypto_provider=crypto_provider
)

## Model Spezifikationen

Hier findet sich das zu verwendende Model. Es handelt sich um ein simples aber [erprobtes Model für MNIST](https://towardsdatascience.com/handwritten-digit-mnist-pytorch-977b5338e627).

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## Trainings und Test Funktionen

Das Training läuft beinahe wie gewohnt ab. Der einzige Unterschied ist, dass der Loss nicht mit der negativen Log-Likelihood Methode (`F.nll_loss` in PyTorch) berechnet werden kann. Dies liegt daran, dass die Funktion schwer mit SMPC realisierbar ist. Stattdessen wird ein simpler "Mean Squared Error"-Loss verwendet.

In [ ]:
def train(args, model, private_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(private_train_loader): # <-- now it is a private dataset
        start_time = time.time()
        
        optimizer.zero_grad()
        
        output = model(data)
        
        # loss = F.nll_loss(output, target)  <-- not possible here
        batch_size = output.shape[0]
        loss = ((output - target)**2).sum().refresh()/batch_size
        
        loss.backward()
        
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            loss = loss.get().float_precision()
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tTime: {:.3f}s'.format(
                epoch, batch_idx * args.batch_size, len(private_train_loader) * args.batch_size,
                100. * batch_idx / len(private_train_loader), loss.item(), time.time() - start_time))
            

Die Test Funktion wird nicht abgeändert.

In [ ]:
def test(args, model, private_test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in private_test_loader:
            start_time = time.time()
            
            output = model(data)
            pred = output.argmax(dim=1)
            correct += pred.eq(target.view_as(pred)).sum()

    correct = correct.get().float_precision()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct.item(), len(private_test_loader)* args.test_batch_size,
        100. * correct.item() / (len(private_test_loader) * args.test_batch_size)))

### Starten des Trainings

Hier eine kurze Zusammenfassung:  
Zuerst wurden alle Model Parameter sicher auf alle Helfer verteilt. Danach wurden alle Parameter und Optimierer auf eine festgelegte Genauigkeit angepasst. Diese mussten nicht sicher aufgeteilt werden, weil sie in diesem Kontext als öffentlich angesehen werden konnten. Jedoch mussten auch sie mit `.fix_precision()` auf die konsistente Verarbeitung der Daten angepasst werden:  
$W \leftarrow W - \alpha * \Delta W$.

In [ ]:
model = Net()
model = model.fix_precision().share(*workers, crypto_provider=crypto_provider, requires_grad=True)

optimizer = optim.SGD(model.parameters(), lr=args.lr)
optimizer = optimizer.fix_precision() 

for epoch in range(1, args.epochs + 1):
    train(args, model, private_train_loader, optimizer, epoch)
    test(args, model, private_test_loader)

Mit nur einem kleinen Teil des MNIST-Datensatzes konnte eine Genauigkeit von etwa 75% mit 100% verschlüsseltem Training erreicht werden!

# 2. Diskussion

Die Mächtigkeit dieses verschlüsselten Trainings wird nun bleuchtet.

## 2.1 Berechnungszeit

Wie leicht zu erkennen war, dauerte das Training deutlich länger als auf lokalen Rohdaten. Im Speziellen dauert eine Iteration über einen Batch mit 64 Elementen etwa 3.2s, während er nur 13ms in reinem PyTorch benötigt. Während es vielleicht wie ein Hindernis erscheint, sollte bedacht werden, dass alles ferngesteuert und in verschlüsselter Weise ablief: keine einzige Datenzeile wurde offengelegt  
Nachgemessen dauert das Berechnen eines einzigen Elements nur 50ms. Die wirkliche Herausforderun besteht darin zu analysieren wann verschlüsseltes Training von Nöten ist und wann einfache verschlüsselte Vorhersagen ausreichen. 50ms für eine Vorhersage zu benötigen ist in einem realen Szenario absolut akzeptabel!

Ein Flaschenhals ist die zeitaufwändige Berechnung der Aktivierungsfunktion: die RELU-Aktivierung mit SMPC ist besonders langwierig, da sie intern Vergleiche und das SecureNN Protokoll verwendet. Wird die RELU-Funktion mit einer Quadratischen-Aktivierung ersetzt, so sinkt die Berechnungszeit von 3.2s auf 1.2s. Solch ein Vorgehen wird in mehreren Papern zu verschlüsselten Berechnungen wie z. B. CryptoNets vorgeschlagen.

Der Kerngedanke sollte sein, nur das Nötige zu verschlüsseln.

## 2.2 Backpropagation mit SMPC

Wie Backpropagation und Gradienten-Updates rein mit Integern erreicht werden kann, mag verwirrend wirken. Dafür wurde ein neuer Syft Tensor names AutogradTensor entwickelt. In diesem Tutorial wurde er regelmäßig verwendet, auch wenn er nicht direkt in Erscheinung trat. Betrachten lässt er sich bei den Model Gewichten:

In [ ]:
model.fc3.bias

Hinzufügen eines Datenpunktes:

In [ ]:
first_batch, input_data = 0, 0
private_train_loader[first_batch][input_data]

Der AutogradTensor ist deutlich zu erkennen. Er ist zwischen der Torch-Hülle und dem FixedPrecisionTensor angesiedelt. Dies deutet darauf hin, dass seine Werte ebenfalls im endlichen Integer Feld liegen. Ziel des AutogradTensors ist es den Berechnungs-Grafen der verschlüsselten Werte zu speichern. Nützlich wird er, sobald `.backward()` aufgerufen wird und der AutogradTensor dann die inkompatiblen PyTorch-Backward-Funktionen durch kompatible Funktionen für verschlüsselte Werte ersetzt. Beispielsweise bezüglich der Multiplikation, welche den Beaver-Triples-Trick anwendet, soll dieser Trick nicht selbst differenziert werden, sondern die einfache Multiplikation rein an sich: $\partial_b (a \cdot b) = a \cdot \partial b$.  
Hier ist nachzulesen, wie die Gradienten berechnet werden:

```python
class MulBackward(GradFunc):
    def __init__(self, self_, other):
        super().__init__(self, self_, other)
        self.self_ = self_
        self.other = other

    def gradient(self, grad):
        grad_self_ = grad * self.other
        grad_other = grad * self.self_ if type(self.self_) == type(self.other) else None
        return (grad_self_, grad_other)
```

In `tensors/interpreters/gradients.py` kann nachgesehen werden, wie die anderen Gradienten implementiert sind.

In Bezug auf den Berechnungs-Grafen bleibt eine Kopie lokal und der Server kann nicht nur den Forward-Anteil, sondern auch den Backward-Anteil koordinieren.

## 2.3 Sicherheitsgarantien

Hier einige Hinweise auf die erreichte Sicherheit: Angriffe die hier betrachtet wurden, sind rein **ehrlich-aber-neugierig**. Das bedeutet, dass ein Angreifer nichts über die Daten innerhalb der Protokolle lernen kann, jedoch beim Abweichen von diesen Protokollen die ihm anvertrauten Anteile verändern und damit die Berechnung sabotieren kann. Die Sicherheit gegen bösartige Angriffe in solchen SMPC Szenarien bleibt weiterhin ein ungelöstes Problem.

Zusätzlich gibt es weitere Bedrohungen aus der realen Welt, auch wenn die Secure Multi-Party Computation die Daten beim Training an sich schützen konnte. Beispielsweise können mit den aus Anfragen (im Kontext von MLaaS) resultierenden Vorhersagen möglicherweise Rückschlüsse auf die zugrundeliegenden Trainings-Datensätze gezogen werden. Im Speziellen gibt es keinen Schutz gegen Zugehörigkeits-Attacken, einer üblichen Attacke auf Machine Learning Services in der festgestellt werden soll, ob ein bestimmtes Element in dem Trainings-Datensatz enthalten war. Außerdem sind weitere Angriffe durch z. B. ungewolltes Auswendiglernen,  Model Inversion oder Model Extraktion weiterhin möglich. 

Eine effiziente Lösung gegen solche Bedrohungen kann "Differential Privacy" sein. Der Ansatz fügt sich nahtlos in die Secure Multi-Parti Computation ein und kann interesante Sicherheitsgarantien gewährleisten. Momentan wird an einigen Implementierungen gearbeitet und hoffentlich steht bald ein erstes Beispiel zur Verfügung.

# Schlussfolgerung

Gezeigt wurde, dass das Training eines Models mit SMPC vom Code her nicht kompliziert sein muss, auch wenn recht komplexe Strukturen verwendet wurden. Mit diesem Wissen im Hinterkopf können nun eigene Projekte analysiert werden, ob verschlüsselte Berechnungen benötigt werden für das Training oder die Evaluation. Wenn auch verschlüsselte Berechnungen im Allgemeinen langsamer sind, so können sie doch verwendet werden um die Gesamtzahl der Berechnungen zu reduzieren. 

Wenn Ihnen dies gefallen hat und Sie sich der Bewegung für mehr vertrauliche und dezentralisierte AI und AI Daten-Lieferkette anschließen möchten, können Sie das folgendermaßen tun.

### PySyft auf GitHub einen Stern geben! 

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos mit Sternen auszuzeichnen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen. 

- [Gib PySyft einen Stern](https://github.com/OpenMined/PySyft)

### Nutze unsere Tutorials auf GitHub!

Wir haben hilfreiche Tutorials erstellt, um ein Verständnis für Federated und Privacy-Preserving Learning zu entwickeln und zu zeigen wie wir die einzelnen Bausteine weiter entwickeln.

- [PySyft Tutorials ansehen](https://github.com/OpenMined/PySyft/tree/master/examples/tutorials)


### Mach mit bei Slack! 

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei! 

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Entwickler zu werden! Sie können jederzeit zur PySyft GitHub Issues-Seite gehen und nach "Projects" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "good first issue" gekennzeichnet sind. 

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebase beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups! 

 - [OpenMined's Open Collective Page](https://opencollective.com/openmined)